# Create Indicators

### SMA

In [0]:
data['SMA'] = data['Adj Close'].rolling(window = 5).mean()

### EMA

In [0]:
def ema(close, periods): 
    # Minimum number of observations in window required to have a value (otherwise result is NA): min_periods
    return close.ewm(span = periods, min_periods = periods).mean() 

### MACD

In [0]:
def macd(close, n_fast, n_slow): 
    ema_fast = ema(close, n_fast) #12
    ema_slow = ema(close, n_slow) #26
    return ema_fast - ema_slow

data['MACD'] = macd(data['Adj Close'], 12, 26)
data['MACD_Line'] = ema(data['Adj Close'], 9)

### RSI

In [0]:
def rsi(close, n):
    up = []
    dn = []
    rsi_list = []
    diff = close.diff(1) 
    for i in range(0,len(close)):
        if diff[i] >0:
            up.append(diff[i])
            dn.append(0)
        elif diff[i] <0:
            dn.append(-diff[i]) # We put the value of average_loss or dn in the rsi, rsi:(0,100)
            up.append(0)
    up = pd.Series(up)
    dn = pd.Series(dn)

    ema_up = ema(up, n)
    ema_dn = ema(dn, n)
    
    ema_up = np.array(ema_up)
    ema_dn = np.array(ema_dn)
    
    
    for j in range(0,len(ema_up)):
        rsi_list.append((100 * ema_up[j]) / (ema_up[j] + ema_dn[j]))
          
    return pd.Series(rsi_list)

    
data['RSI'] = rsi(data['Adj Close'], 14)

### Stochastic Oscillator

In [0]:
def so(high, low, close, n):
    low_n = low.rolling(window = n).min() # minimum of low price within 14 days
    high_n = high.rolling(window = n).max() # maximum of high price within 14 days
    return 100*(close - low_n)/(high_n - low_n)

data['SO'] = so(data['High'], data['Low'], data['Adj Close'], 14)

#Create the "%D" column in the DataFrame
data['%D'] = data['SO'].rolling(window = 3).mean()

### Bollinger Bands

In [0]:
def bb(close, n, ndev):
    mavg = close.rolling(window = n).mean()
    mstd = close.rolling(window = n).std()
    hband = mavg + ndev * mstd
    lband = mavg - ndev * mstd
    return mavg, hband, lband

data['BB_mavg'], data['BB_H'], data['BB_L'] = bb(data['Adj Close'], n = 20, ndev = 2)

### Williams %R

In [0]:
def wr(close, high, low, n):
    high_n = high.rolling(window = n).max() # maximum of high price within 14 days
    low_n = low.rolling(window = n).min() # minimum of low price within 14 days
    return -100*(high_n - close)/(high_n - low_n)

data['%R'] = wr(data['Adj Close'], data['High'], data['Low'], 14)

### PROC

In [0]:
def proc(close, n):
    close_pre = close.shift(n)
    proc = 100*(close - close_pre)/close_pre
    return proc

data['PROC'] = proc(data['Adj Close'], 12)

### Log Return

In [0]:
data['Log'] = np.log(data['Adj Close']) - np.log(data['Adj Close'].shift(1))

### OBV

In [0]:
def obv(close, volume):
    obv_pre = volume[0]
    obv = [obv_pre]
    for i in range(1,len(data)):
        if close[i] > close[i-1]:
            obv_pre = obv_pre + volume[i]
            obv.append(obv_pre)
        elif close[i] == close[i-1]:
            obv_pre = obv_pre + 0
            obv.append(obv_pre)
        elif close[i] < close[i-1]:
            obv_pre = obv_pre - volume[i]
            obv.append(obv_pre)

    return pd.Series(obv)

data['OBV'] = obv(data['Adj Close'], data['Volume'])